In [ ]:
#conda env export > environment.yml

replace contractions/location abbreviations via dictionary
denoise text/apply word segmentation
create word count dictionary
next, spell check:
add to dictionary from reoccuring words in text? or use freq dict symspell
apply spell check to words in word count dictionary occuring only one time
apply pos tag to tweets creating new column
lemmatize tweets based on pos tag column
bigrams with Phraser

In [1]:
import os
import re
import string
import pandas as pd 
import itertools

## import datasets

In [290]:
# import dataset
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
#target = pd.read_csv('train.csv', usecols=['target'])

In [3]:
print(len(df))

7613


In [279]:
print(len(test))

3263


In [4]:
#test_df.drop(['id'], axis=1, inplace=True)
df.drop(['id'], axis=1, inplace=True)

#dups = df[df.duplicated(['text'])]
df.drop_duplicates(subset=['text', 'location', 'keyword'], keep='first', inplace=True)
#test_df.drop_duplicates(subset=['text'], keep='first', inplace=True)

print(len(df))
#print(len(test_df))

## visualize target data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline
from jupyterthemes import jtplot
jtplot.style(theme='solarizedl')

In [ ]:
# visualize distribution of target in dataset
plt.figure(figsize=(11,11))
colors = ['lightblue', 'red']
expl = (0, 0.1)
df.target.value_counts().plot(kind='pie', legend=True, startangle=45, shadow=True, 
                             colors=colors, autopct='%1.1f%%')
plt.title('Binary Distribution of Disaster Tweet Dataset', fontsize=20)

# clean and process text

In [6]:
from symspellpy.symspellpy import SymSpell, Verbosity
from spellchecker import SpellChecker
import pkg_resources

## hashtags

In [7]:

# functions related to expanded hashtags in text
def pascal_case_split(text):
    # expand hashtags formatted in pascal case, ex: #ThisIsPascal
    return re.sub(r'([A-Z])([?=a-z0-9+])', r' \1\2', text)

def camel_case_split(text):
    # expand hashtags formatted in pascal case, ex: #thisIsCamel
    return re.sub(r'([a-z0-9+])([?<=A-Z])', r'\1 \2', text)

def spell_check_word(word):
    # lookup suggestions for individual words
    spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=5)
    dictionary_path = pkg_resources.resource_filename(
                      "symspellpy", "frequency_dictionary_en_82_765.txt")
    spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
    corrections = spell.lookup(word, Verbosity.TOP, include_unknown=True)
    closest = corrections[0]
    # display suggestion term, edit distance, and term frequency
    return closest.term
    
def spell_check_words(word):
    # lookup suggestions for multi word string input
    spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=5)
    dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
    bigram_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_bigramdictionary_en_243_342.txt")
    spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
    spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)
    corrections = spell.lookup_compound(word, max_edit_distance=2)
    closest = corrections[0]
# display suggestion term, edit distance, and term frequency
    return closest.term

def desegment_strings(text):
    spell = SymSpell(max_dictionary_edit_distance=0, prefix_length=8)
    dictionary_path = pkg_resources.resource_filename("symspellpy","frequency_dictionary_en_82_765.txt")
    spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
    if text.isupper() | text.islower() and text.startswith('#'):
        result = spell.word_segmentation(text)
        text = result.corrected_string
    return text 

def remove_hash(text):
    # remove hash symbol in front of hashtag text and remove non unicode chars
    return re.sub('#', '', text)

def expand_strings(text):
    # desegment hashtags & other strings with similar formatting
    text = re.sub('CAfire', 'california fire', text)
    c_text = camel_case_split(text)
    p_text = pascal_case_split(c_text)
    h_text = remove_hash(p_text)
    return h_text

def expand_strings_sc(text):
    # combine hashtag split functions for specific cases and remove hash, spell check using compound lookup
    c_text = camel_case_split(text)
    p_text = pascal_case_split(c_text)
    lu_text = spell_check_words(p_text)
    h_text = remove_hash(lu_text)
    return h_text

def expand_strings_ds(text):
    # combine hashtag split functions for specific cases and remove hash. spell check using wordsegmentation
    c_text = camel_case_split(text)
    p_text = pascal_case_split(c_text)
    lu_text = desegment_strings(p_text)
    h_text = remove_hash(lu_text)
    return h_text

def expand_strings_indv_word(text):
    # combine hashtag split functions for specific cases and remove hash. spell check using wordsegmentation
    c_text = camel_case_split(text)
    p_text = pascal_case_split(c_text)
    lu_text = spell_check_word(p_text)
    h_text = remove_hash(lu_text)
    return h_text

# def expand_strings_ds(text):
#     # combine hashtag split functions for specific cases and remove hash
#     c_text = camel_case_split(text)
#     p_text = pascal_case_split(c_text)
#     lu_text = desegment_hashtag(p_text)
#     h_text = remove_hash(lu_text)
#     return h_text

In [ ]:
# create column for extracted hashtags
df['hashtags'] = df.text.str.findall(r'(?:(?<=\s)|(?<=^))#.*?(?=\s|$)')
#test_df['hashtags'] = test_df.text.str.findall(r'(?:(?<=\s)|(?<=^))#.*?(?=\s|$)')
df.hashtags = df.hashtags.astype(str)
#test_df.hashtags = test_df.hashtags.astype(str)

%time
df['exp_hash'] = df['hashtags'].map(expand_hashtags)

In [ ]:
df.location.unique()


%time
# with desegment_hashtag
df['exp_hash'] = df['hashtags'].map(expand_strings)

# process text

## dictionary based word replacement 

In [8]:
import csv

In [9]:
# import and read previously created dictionary as csv for abbreviations and expansions of words
def csv2dict(csv_name):
    with open(csv_name, mode='r') as f:
        reader = csv.reader(f)
        next(reader)
        return {rows[1]:rows[2] for rows in reader}
location_expansion_dict = csv2dict('utils\location_expansion.csv')
contractions_dict = csv2dict('utils\contractions.csv')
internet_initialisms_dict = csv2dict('utils\internet_initialisms.csv')

## denoise text functions

In [10]:
import html

In [11]:
def word_expansion_from_dict(text, expansion_dict):
    c_re = re.compile(r'\b'+r'\b|\b'.join(map(re.escape, expansion_dict.keys()))+r'\b')
    def replace(match):
        expansion =  f"{expansion_dict[match.group(0)]}"
        return expansion
    text = c_re.sub(replace, text)
    return text

def reduce_repeat_vowels(text, keep_reps=1):
    # function to reduce repeated vowel occurences in string, 
    # keep_reps argument as int refers to number of occurences to keep in string
    # keep_reps argument default : 1, can be set to any integer
    vowels = ['a','e','i','o','u']
    for ch in iter(vowels):
        text = re.sub(r'(?i)'+rf'([{ch}])'+r'{3,}', r'\1'*int(keep_reps), text)
    return text

def reduce_repeat_consonants(text, keep_reps=2):
    # function to reduce repeated consonant occurences in string, 
    # keep_reps argument as int refers to number of occurences to keep in string
    # keep_reps argument default : 1, can be set to any integer
    consonants = ['b','c','d','f','g','h','j','k','l','m','n','p','q','r','s','t','v','w','x','y','z']
    for ch in iter(consonants):
        text = re.sub(r'(?i)'+ rf'([{ch}])'+r'{3,}', r'\1'*int(keep_reps), text)
    return text
    
def remove_urls(text):
    # function to remove urls from string
    return re.sub(r"http\S+",  r"", text)

def remove_handles(text):
    # function to remove twitter handles from string
    return re.sub("@\S*",  r" ", text)

def remove_non_uni(text):
    # function to remove non unicode characters from string
    return re.sub('[^\u0000-\u007f]', ' ', text)

def remove_nan_str(text):
    # function to remove 'nan' from string
    return re.sub('nan', '', text)

def remove_ws(text):
    return re.sub(' +', ' ', text)

def remove_non_alpha(text):
    # function to remove non alpha chars from string
    return re.sub("[^a-zA-Z]",  r" ", text)

def denoise_text(text):
    #reduce repeating characters
    text = reduce_repeat_vowels(text)
    text = reduce_repeat_consonants(text)
    #remove xml tag strings
    text = html.unescape(text)
    #remove numeric and punctuation chars
    text = remove_non_alpha(text)
    #remove extra spaces
    text = remove_ws(text)
    return text   
    

## tweet text denoise 

In [12]:
#### text preprocessing specifically formatted for tweets but will work on any text
def process_tweet(df): 
    """combine regex and nltk processing for tweet text and keyword text processing"""
    def expand_strings_ds(text):
        # combine hashtag split functions for specific cases and remove hash
        # remove urls and usernames first due to alternating casing inside before case format expansion
        u_text = remove_urls(text)
        hd_text = remove_handles(u_text)
        c_text = camel_case_split(hd_text)
        p_text = pascal_case_split(c_text)
        lu_text = desegment_strings(p_text)
        h_text = remove_hash(lu_text)
        return h_text
    
    def expand_text(text):
        e_text = expand_strings_ds(text)
        le_text = word_expansion_from_dict(e_text, expansion_dict=location_expansion_dict)
        ce_text = word_expansion_from_dict(le_text.lower(), expansion_dict=contractions_dict)
        ii_text = word_expansion_from_dict(ce_text.lower(), expansion_dict=internet_initialisms_dict)
        return ii_text

    def process_text(text):
        exp_text = expand_text(text)
        clean_text = denoise_text(exp_text)
        #lem_text = lemmatize_text(clean_text)
        return clean_text

    processed_df = [process_text(x) for x in df]
    return processed_df

## keyword text denoise process

In [13]:
def process_keyword(df):
    text = df.str.replace('%20', ' ')
    return text

## location text denoise process

In [391]:
location_expansion_dict

{'AK': 'Alaska',
 'AL': 'Alabama',
 'AR': 'Arkansas',
 'AZ': 'Arizona',
 'CA': 'California',
 'CO': 'Colorado',
 'CT': 'Connecticut',
 'DC': 'District of Columbia',
 'DE': 'Delaware',
 'FL': 'Florida',
 'GA': 'Georgia',
 'GU': 'Guam',
 'HI': 'Hawaii',
 'IA': 'Iowa',
 'ID': 'Idaho',
 'IL': 'Illinois',
 'KS': 'Kansas',
 'KY': 'Kentucky',
 'LA': 'Louisiana',
 'MA': 'Massachusetts',
 'MD': 'Maryland',
 'ME': 'Maine',
 'MI': 'Michigan',
 'MN': 'Minnesota',
 'MO': 'Missouri',
 'MP': 'Northern Mariana Islands',
 'MS': 'Mississippi',
 'MT': 'Montana',
 'NC': 'North Carolina',
 'ND': 'North Dakota',
 'NE': 'Nebraska',
 'NH': 'New Hampshire',
 'NJ': 'New Jersey',
 'NM': 'New Mexico',
 'NV': 'Nevada',
 'NY': 'New York',
 'OH': 'Ohio',
 'OK': 'Oklahoma',
 'OR': 'Oregon',
 'PA': 'Pennsylvania',
 'PR': 'Puerto Rico',
 'PW': 'Palau',
 'RI': 'Rhode Island',
 'SC': 'South Carolina',
 'SD': 'South Dakota',
 'TN': 'Tennessee',
 'TX': 'Texas',
 'UT': 'Utah',
 'VA': 'Virginia',
 'VI': 'Virgin Islands',
 'V

In [395]:
def process_location(df):
    """combine regex and nltk processing for location text processing"""
    
    def expand_loc_text(text):
        ex_text = word_expansion_from_dict(text, location_expansion_dict)
        return ex_text
       
        
    def process_text(text):
        text = expand_loc_text(text)
        clean_text = denoise_text(text)
        #lem_text = lemmatize_text(clean_text)
        return clean_text
    
    loc_df = [process_text(x) for x in df]
    return loc_df 

# create new dataframe of denoised text

In [15]:
def scrub_data_to_df(df):
    df['tweets'] = process_tweet(df.text)
    df['clean_keyword'] = process_keyword(df.keyword.astype(str))
    df['clean_location'] = process_location(df.location.astype(str))
    return df

In [16]:
train_df = scrub_data_to_df(df)
test_df = scrub_data_to_df(test)
all_tweets = pd.DataFrame()
all_tweets['tweets'] = pd.concat([train_df.tweets, test_df.tweets], ignore_index=True)
all_tweets['clean_keyword'] = pd.concat([train_df.clean_keyword, test_df.clean_keyword], ignore_index=True)
all_tweets['clean_location'] = pd.concat([train_df.clean_location, test_df.clean_location], ignore_index=True)

In [17]:
all_tweets.to_csv('complete_tweets.csv')

## create frequency dictionary for tweet text for further spell corrections

In [19]:
from collections import Counter
from operator import itemgetter

In [134]:
def get_corpus(df):
    tweets = all_tweets.split()
    corpus = ' '.join(word for word in tweets)
    return corpus

def word_count(text):
    counts = dict()
    for row in text:
        words = row.split()
        for w in words:
            if w in counts:
                counts[w] += 1
            else:
                counts[w] = 1

    return counts

In [138]:
import nltk
#tokens = all_tweets.tweets.astype(str).split()
# count frequency of words in corpus
word_count_dict = word_count(all_tweets.tweets)

# count number of words in corpus
num_words = sum(word_count_dict[w] for w in word_count_dict)
print(f'There are {num_words} words in the entire training dataset')

# count number of unique words in corpus
word_count_sorted = [(value, key) for key, value in word_count_dict.items()]
word_count_sorted.sort()
print(f'There are {len(word_count_sorted)} unique words in the training dataset')

single_occ_words =  [k for k,v in word_count_dict.items() if int(v) == 1]
filtered_dict = {k:v for (k,v) in word_count_dict.items() if int(v)>=4}

There are 153365 words in the entire training dataset
There are 16645 unique words in the training dataset


In [161]:
def spell_check_word(word):
    # lookup suggestions for individual words
    spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=5)
    dictionary_path = pkg_resources.resource_filename(
                      "symspellpy", "frequency_dictionary_en_82_765.txt")
    spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
    corrections = spell.lookup(word, Verbosity.TOP, include_unknown=True)
    closest = corrections[0]
    # display suggestion term, edit distance, and term frequency
    return closest.term

In [162]:
def spell_check_rare(df):
    corrections = []
    for x in single_occ_words:
        repl = spell_check_word(x)
        corrections.append(repl)
    replacement_dict = dict(zip(single_occ_words, corrections))
    filtered_repl = dict(filter(lambda elem: elem[0] != elem[1], replacement_dict.items()))
    def replace_spell(text):
        r_re = re.compile(r'\b'+r'\b|\b'.join(map(re.escape, filtered_repl.keys()))+r'\b')
        def replace(match):
            replacement =  f"{filtered_repl[match.group(0)]}"
            return replacement
        text = r_re.sub(replace, text)
        return text
    all_tweets['sc'] = all_tweets['tweets'].apply(replace_spell)
    return df

In [163]:
spell_check_rare(all_tweets)

,tweets,new,sc
0,our deeds are the reason of this earthquake m...,our deeds are the reason of this earthquake m...,our deeds are the reason of this earthquake m...
1,forest fire near la ronge sask canada,forest fire near la range ask canada,forest fire near la range sask canada
2,all residents asked to shelter in place are b...,all residents asked to shelter in place are b...,all residents asked to shelter in place are b...
3,people receive wildfires evacuation orders in...,people receive wildfires evacuation orders in...,people receive wildfires evacuation orders in...
4,just got sent this photo from ruby alaska as ...,just got sent this photo from ruby alaska as ...,just got sent this photo from ruby alaska as ...
...,...,...,...
10871,earthquake safety los angeles safety fasteners...,earthquake safety los angeles safety fasteners...,earthquake safety los angeles safety fasteners...
10872,storm in rhode island worse than last hurrica...,storm in rhode island worse than last hurrica...,storm in rhode island worse than last hurrica...
10873,green line derailment in chicago,green line derailment in chicago,green line derailment in chicago
10874,meg issues hazardous weather outlook hwo,meg issues hazardous weather outlook hwo,meg issues hazardous weather outlook hwo


## tokenize text

In [189]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag, pos_tag_sents, sent_tokenize

In [226]:
def tokenize_text(text):
    tokens = []
    stopword_list = stopwords.words('english')
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if word not in stopword_list:
                tokens.append(word.lower())
    return tokens



def nltk_tag_to_wordnet_tag(nltk_tag):
# function to convert nltk tag to wordnet tag
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None
    
def lemmatize_text(text):
    # define lemmatizer
    lemmatizer = WordNetLemmatizer()
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(tokenize_text(text))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    # initialize empty list 
    lemmatized_text = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_text.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_text.append(lemmatizer.lemmatize(word, tag))
    return ' '.join(x for x in lemmatized_text if len(x) > 1)


def remove_stopwords(tokens):
    # function to remove stopwords from text
    stop_list = stopwords.words('english')  
    new_tokens = []
    for word in tokens:
        if word not in stop_list:
            new_tokens.append(word)
    return new_tokens


In [227]:
#### text preprocessing specifically formatted for tweets but will work on any text
def tokenize_tweet(text): 
    """combine regex and nltk processing for tweet text and keyword text processing"""
    lem_text = lemmatize_text(text)
    return lem_text

In [197]:
all_tweets['tokens'] = all_tweets.tweets.apply(tokenize_text)
all_tweets['stems'] = all_tweets.tokens.apply(lambda row: [stemmer.stem(item) for item in row]) 
all_tweets['lem'] = all_tweets.tweets.apply(lemmatize_text)
all_tweets.to_csv('tweets_processed.csv')

## separate test/train data

In [ ]:
# split all_tweets back into 2 separate datasets & save
train_clean = all_tweets[:7613]
test_clean = all_tweets[7613:]

train_clean.to_csv('train_clean.csv')
test_clean.to_csv('test_clean.csv')

In [485]:
# import og train dataset to add target to train data before dropping duplicates
tr = pd.read_csv('train.csv')
te = pd.read_csv('test.csv')

# import and read from csvs just created so df is no longer a slice of all_tweets
train_clean = pd.read_csv('train_clean.csv', usecols=["tweets", "tokens", "stems", "lem", "clean_location", "clean_keyword"])
test_clean = pd.read_csv('test_clean.csv', usecols=["tweets", "tokens", "stems", "lem", "clean_location", "clean_keyword"])
# add id & target columns from og data to dataframes, no target available for test data
train_clean['target'] = tr.target
train_clean['id'] = tr.id
test_clean['id'] = te.id
# drop duplicates from each dataset 
test_clean.drop_duplicates(subset=['tweets'], keep='first', inplace=True)
train_clean.drop_duplicates(subset=['tweets'], keep='first', inplace=True)

In [486]:
print(len(train_clean))
print(len(test_clean))

6830
3064


In [487]:
comp_df = pd.concat([train_clean, test_clean], ignore_index=True)

## find and notate recurring bigrams from tweets

In [488]:
def join_tokens(tokens):
    joined_tweet = re.sub(r'[^a-zA-Z]', r' ', str(tokens))
    return joined_tweet 
comp_df['joined_tokens'] = join_tokens(comp_df.tokens)
comp_df['joined_stems'] = join_tokens(comp_df.stems)

In [489]:
from gensim.models.phrases import Phrases, Phraser, ENGLISH_CONNECTOR_WORDS

def sentence_to_bi_grams(sentence):
    return ' '.join(phrases[sentence])
texts = comp_df.lem.astype(str)
def get_documents(text):
    docs = [row.split() for row in text]
    return docs
doc = get_documents(texts)

phrases = Phrases(doc, min_count=5, threshold=.75)
frozen_phrases = phrases.freeze()

bigrams = []
for row in doc: 
    parsed_sentence = sentence_to_bi_grams(row)
    bigrams.append(parsed_sentence)
    
comp_df['bigrams'] = bigrams

In [490]:
# split all_tweets back into 2 separate datasets & save for classification process
train_final = comp_df[:6830]
test_final = comp_df[6830:]
train_final.to_csv('train_final.csv')
test_final.to_csv('test_final.csv')